In [1]:
!set HTTP_PROXY=127.0.0.1:7890
!set HTTPS_PROXY=127.0.0.1:7890
#!set CUDA_HOME=C:\Program Files\NVIDIAGPUComputingToolkit\CUDA\v12.5

In [ ]:
!pip uninstall -y transformers
!pip install numpy==1.24.4 
!pip install Pillow Requests==2.31.0 
!pip install torch==2.3.0 torchvision==0.18.0 --index-url https://download.pytorch.org/whl/cu121
!pip install Requests==2.31.0 transformers==4.40.2
#!pip install flash_attn==2.5.8 

In [ ]:
!pip install accelerate

In [ ]:
#!pip list 
!nvcc -V
!ninja --version
!echo $?

In [5]:
import torch

from transformers.utils  import is_flash_attn_2_available
print(is_flash_attn_2_available())


False


In [6]:
from PIL import Image
import requests
from transformers import AutoModelForCausalLM
from transformers import AutoProcessor


In [7]:
model_id = 'microsoft/Phi-3-vision-128k-instruct'

In [ ]:
#import torch

model = AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda", trust_remote_code=True, torch_dtype="auto")

In [ ]:
processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)


In [ ]:
messages = [
    {"role": "user", "content": "<|image_1|>\nWhat is shown in this image?"},
    {"role": "assistant", "content": "The chart displays the percentage of respondents who agree with various statements about their preparedness for meetings. It shows five categories: 'Having clear and pre-defined goals for meetings', 'Knowing where to find the information I need for a meeting', 'Understanding my exact role and responsibilities when I'm invited', 'Having tools to manage admin tasks like note-taking or summarization', and 'Having more focus time to sufficiently prepare for meetings'. Each category has an associated bar indicating the level of agreement, measured on a scale from 0% to 100%."},
    {"role": "user", "content": "Provide insightful questions to spark discussion."}
]

In [ ]:
url = "https://assets-c4akfrf5b4d3f4b7.z01.azurefd.net/assets/2024/04/BMDataViz_661fb89f3845e.png"


In [ ]:
image = Image.open(requests.get(url, stream=True).raw)

In [ ]:
image

In [ ]:
prompt = processor.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)


In [ ]:
inputs = processor(prompt, [image], return_tensors="pt").to("cuda:0")


In [ ]:
generation_args = {
    "max_new_tokens": 500,
    "temperature": 0.0,
    "do_sample": False,
}

In [ ]:
generate_ids = model.generate(**inputs, eos_token_id=processor.tokenizer.eos_token_id, **generation_args)
